# Import Bibliotheken

In [136]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [137]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [138]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [139]:
comment_df = pd.read_excel('../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [140]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [141]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [142]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [143]:
# next(iter(train_set))

## Dataloader

### Tokenisierung

In [144]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [145]:
# Beispiel für Anwendung des tokenizer
# sentence = "This isn't a very long example."
# tokenizer(sentence)

### Erstellung Vokabular

In [146]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [147]:
# Länge des Trainungsets
len(train_set)

2798

In [148]:
tokens = yield_tokens(train_set)

In [149]:
next(iter(tokens))

['sauber', 'und', 'ordentlich', 'passt']

In [150]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [151]:
len(vocab)

2392

In [152]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 4, 0]

In [153]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [154]:
onehot = torch.zeros(1,len(vocab))

In [155]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[35, 0, 46]

In [156]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [157]:
import torch
import torch.nn

In [158]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [159]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [160]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [161]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [162]:
unflat = [[x] for x in flat_set]

In [163]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1,)

label_vocab.set_default_index(vocab["<unk>"])

In [164]:
label_vocab.lookup_token(3)

'E-Fuels'

#### Dictionary erstellen

In [165]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Personal': 0,
 'Bistro': 1,
 'Paketservice': 2,
 'Werkstatt': 3,
 'Nacht-/Tankautomat': 4,
 'E-Mobilität': 5,
 'Waschanlage': 6,
 'Zapfsäulen': 7,
 'Sanitär': 8,
 'Kraftstoffauswahl': 9,
 'Shop': 10,
 'Verkehrsanbindung': 11,
 'DigitalFueling': 12,
 'Tankpool': 13,
 'E-Fuels': 14,
 'SB-Waschboxen': 15,
 'Pricing': 16,
 'Erscheinungsbild': 17,
 'Kartenakzeptanzen': 18,
 'WLAN': 19,
 'Öffnungszeiten': 20,
 'Staubsauger': 21,
 'Luft': 22,
 'AdBlue': 23}

#### Tensor

In [166]:
doc2 = label_list_export[2]
doc2

['Personal', 'Waschanlage']

In [167]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[0, 6]

In [168]:
labels = torch.LongTensor(pos_vals)
labels

tensor([0, 6])

In [169]:
y_onehot = nn.functional.one_hot(labels, num_classes=24)
y_onehot

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [170]:
y_onehot = y_onehot.sum(dim=0).float()
y_onehot

tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [171]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

### Erstellung Dataloader

In [172]:
# label_pipeline = lambda x: int(x) -1

In [173]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [174]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [175]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [176]:
vocab_size = len(vocab)
num_class = 24
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [177]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.03

## Initialisierung Fehlerfunktion
loss_fn = nn.CrossEntropyLoss()

## Initialisierung Fehlerfunktion
bce_loss = nn.BCEWithLogitsLoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 150

In [178]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold= 0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [179]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 0, 0,

In [180]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.29235169291496277
Training Accuracy for epoch 1:0.3137955665588379
Training Accuracy for epoch 2:0.37097927927970886
Training Accuracy for epoch 3:0.38384559750556946
Training Accuracy for epoch 4:0.3877769708633423
Training Accuracy for epoch 5:0.38920658826828003
Training Accuracy for epoch 6:0.3899213671684265
Training Accuracy for epoch 7:0.39135095477104187
Training Accuracy for epoch 8:0.3945675492286682
Training Accuracy for epoch 9:0.39206576347351074
Training Accuracy for epoch 10:0.3945675492286682
Training Accuracy for epoch 11:0.38920658826828003
Training Accuracy for epoch 12:0.39385274052619934
Training Accuracy for epoch 13:0.39135095477104187
Training Accuracy for epoch 14:0.38170120120048523
Training Accuracy for epoch 15:0.3906361758708954
Training Accuracy for epoch 16:0.392423152923584
Training Accuracy for epoch 17:0.3917083740234375
Training Accuracy for epoch 18:0.39099356532096863
Training Accuracy for epoch 19:0.3795568346977234


In [181]:
batch_train_accuracy

tensor(0.4214)

### Auswertung Fehlerentwicklung

In [182]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.691167
1,1,0.687049
2,2,0.684334
3,3,0.683539
4,4,0.683296


In [183]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
).properties(title='Trainingsfehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [184]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.292352
1,1,0.313796
2,2,0.370979
3,3,0.383846
4,4,0.387777


In [185]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
).properties(title='Trainingsgenauigkeit in Abhängigkeit von der Anzahl der Epochen')
accuracy_chart

alt.Chart(...)

## Evaluierung

In [186]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2392, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=24, bias=True)
)

### Gesamtevaluierung

In [187]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [188]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [189]:
total_valid_accuracy

tensor(0.3970)

### Evaluierung nach Klassen

In [191]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=24
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [192]:
total_valid_accuracy

tensor([0.5721, 0.8240, 0.9958, 0.9992, 0.9967, 0.8557, 0.9591, 0.9374, 0.9758,
        0.9733, 0.9900, 0.8616, 0.8098, 0.9842, 0.9641, 0.8791, 0.9892, 0.9992,
        0.9633,    nan, 0.8932, 0.9917, 0.9850, 0.9750])

### Evaluierung nach der Anzahl der Epochen

In [193]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [194]:
test_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [196]:
vocab_size = len(vocab)
num_class = 24
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [197]:
bce_loss = nn.BCEWithLogitsLoss()

In [198]:
learning_rate = 0.03

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [199]:
num_epochs = 150

In [200]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [201]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = bce_loss(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()



Training Accuracy for epoch 0: 0.2730521857738495
Validation Accuracy for epoch 0: 0.28734809160232544
Training Accuracy for epoch 1: 0.3027162253856659
Validation Accuracy for epoch 1: 0.3016440272331238
Training Accuracy for epoch 2: 0.3091493844985962
Validation Accuracy for epoch 2: 0.31629735231399536
Training Accuracy for epoch 3: 0.3080771863460541
Validation Accuracy for epoch 3: 0.3062902092933655
Training Accuracy for epoch 4: 0.31343817710876465
Validation Accuracy for epoch 4: 0.3184417486190796
Training Accuracy for epoch 5: 0.35132238268852234
Validation Accuracy for epoch 5: 0.3784846365451813
Training Accuracy for epoch 6: 0.372051477432251
Validation Accuracy for epoch 6: 0.3777698278427124
Training Accuracy for epoch 7: 0.3795568346977234
Validation Accuracy for epoch 7: 0.38563260436058044
Training Accuracy for epoch 8: 0.38170120120048523
Validation Accuracy for epoch 8: 0.38170120120048523
Training Accuracy for epoch 9: 0.38205862045288086
Validation Accuracy for e

In [202]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "validation"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [203]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
 x='Epoch',
 y='Loss',
 color='Type'
).properties(title='Fehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_eval_loss_chart

alt.Chart(...)

In [204]:
valid_accuracy_hist

{0: array(0.2873481, dtype=float32),
 1: array(0.30164403, dtype=float32),
 2: array(0.31629735, dtype=float32),
 3: array(0.3062902, dtype=float32),
 4: array(0.31844175, dtype=float32),
 5: array(0.37848464, dtype=float32),
 6: array(0.37776983, dtype=float32),
 7: array(0.3856326, dtype=float32),
 8: array(0.3817012, dtype=float32),
 9: array(0.3849178, dtype=float32),
 10: array(0.3849178, dtype=float32),
 11: array(0.38741958, dtype=float32),
 12: array(0.38599, dtype=float32),
 13: array(0.39170837, dtype=float32),
 14: array(0.39563975, dtype=float32),
 15: array(0.39599714, dtype=float32),
 16: array(0.39206576, dtype=float32),
 17: array(0.39742672, dtype=float32),
 18: array(0.39742672, dtype=float32),
 19: array(0.39706933, dtype=float32),
 20: array(0.39528236, dtype=float32),
 21: array(0.39814153, dtype=float32),
 22: array(0.39599714, dtype=float32),
 23: array(0.39063618, dtype=float32),
 24: array(0.39027876, dtype=float32),
 25: array(0.39528236, dtype=float32),
 26: 

In [205]:
train_accuracy_hist

{0: array([0.57214344, 0.82402   , 0.9958299 , 0.99916595, 0.99666387,
        0.85571307, 0.9591326 , 0.93744785, 0.97581315, 0.97331107,
        0.98999166, 0.8615513 , 0.8098415 , 0.98415345, 0.9641368 ,
        0.8790659 , 0.9891576 , 0.99916595, 0.96330273,        nan,
        0.8932444 , 0.9916597 , 0.9849875 , 0.97497916], dtype=float32),
 1: array(0.2873481, dtype=float32),
 2: array(0.30164403, dtype=float32),
 3: array(0.31629735, dtype=float32),
 4: array(0.3062902, dtype=float32),
 5: array(0.31844175, dtype=float32),
 6: array(0.37848464, dtype=float32),
 7: array(0.37776983, dtype=float32),
 8: array(0.3856326, dtype=float32),
 9: array(0.3817012, dtype=float32),
 10: array(0.3849178, dtype=float32),
 11: array(0.3849178, dtype=float32),
 12: array(0.38741958, dtype=float32),
 13: array(0.38599, dtype=float32),
 14: array(0.39170837, dtype=float32),
 15: array(0.39563975, dtype=float32),
 16: array(0.39599714, dtype=float32),
 17: array(0.39206576, dtype=float32),
 18: ar

In [206]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [207]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "validation"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [208]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.287348,train
1,2,0.301644,train
2,3,0.316297,train
3,4,0.306290,train
4,5,0.318442,train
...,...,...,...
144,145,0.399929,validation
145,146,0.399929,validation
146,147,0.399929,validation
147,148,0.399214,validation


In [209]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
 x='Epoch',
 y='Accuracy',
 color='Type'
).properties(title='Genauigkeiten des Trainings- und Testdatensatzes nach der Anzahl der Epochen')
train_eval_accuracy_chart

alt.Chart(...)